In [3]:
import secrets
import string

def generate_strong_password(length=33):
    characters = string.ascii_letters + string.digits + string.punctuation
    password = ''.join(secrets.choice(characters) for _ in range(length))
    return password

# Generate a 16-character strong password
print(generate_strong_password(33))

_-U@Jjr3VG]^'nscz7a)gW+r[B,/Tlu+A
